In [ ]:
import numpy as np
import pandas as pd

DATA_PATH = "C:/Users/NataliaUDC/Downloads/higgs/HIGGS.csv.gz"

# 1. Nombres de columnas: 1 etiqueta + 28 features reales
col_names = ["label"] + [f"f{i}" for i in range(28)]

# 2. Cargar el dataset local
higgs = pd.read_csv(
    DATA_PATH,
    compression="gzip" if DATA_PATH.endswith(".gz") else None,
    header=None,
    names=col_names
)

print("Tamaño total del dataset:", higgs.shape)

# 3. División: train inicial, test final, extra intermedio
N_TRAIN = 2_000_000
N_TEST = 500_000

train_df = higgs.iloc[:N_TRAIN]
test_df  = higgs.iloc[-N_TEST:]
extra_df = higgs.iloc[N_TRAIN:-N_TEST]

print("Train:", train_df.shape)
print("Test:",  test_df.shape)
print("Extra:", extra_df.shape)

# 4. Separar features (X) y labels (y)
def split_xy(df):
    X = df.drop(columns="label").to_numpy(dtype=np.float32)
    y = df["label"].to_numpy(dtype=np.float32)
    return X, y

X_train, y_train = split_xy(train_df)
X_test,  y_test  = split_xy(test_df)
X_extra, y_extra = split_xy(extra_df)

Tamaño total del dataset: (11000000, 29)
Train: (2000000, 29)
Test: (500000, 29)
Extra: (8500000, 29)


In [4]:
# 5. Normalizar / estandarizar "a mano" usando solo TRAIN

# Media y desviación por columna (feature) calculadas a partir del train
mean_train = X_train.mean(axis=0)
std_train  = X_train.std(axis=0)

# Evitar división por cero
std_train[std_train == 0] = 1.0

# Aplicar la transformación estándar: (x - media) / sd
X_train = (X_train - mean_train) / std_train
X_extra = (X_extra - mean_train) / std_train
X_test  = (X_test  - mean_train) / std_train

# 6. Comprobación del desbalanceo de clases
def class_stats(name, y):
    unique, counts = np.unique(y, return_counts=True)
    print(f"\nDistribución de clases en {name}:")
    for cls, cnt in zip(unique, counts):
        print(f"  Clase {int(cls)} → {cnt} ejemplos ({cnt / len(y):.3%})")

class_stats("Train", y_train)
class_stats("Extra", y_extra)
class_stats("Test",  y_test)



Distribución de clases en Train:
  Clase 0 → 941182 ejemplos (47.059%)
  Clase 1 → 1058818 ejemplos (52.941%)

Distribución de clases en Extra:
  Clase 0 → 3994202 ejemplos (46.991%)
  Clase 1 → 4505798 ejemplos (53.009%)

Distribución de clases en Test:
  Clase 0 → 235493 ejemplos (47.099%)
  Clase 1 → 264507 ejemplos (52.901%)


In [ ]:
# CON MÉTODOS DE LIBRERÍA 
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('HIGGS.csv.gz', header=None, compression='gzip')

print(f"Dataset: {df.shape[0]} ejemplos, {df.shape[1]} columnas")

# Separar características (X) y etiquetas (y)
X = df.iloc[:, 1:].values  # Features (columnas 1-28)
y = df.iloc[:, 0].values   # Labels (columna 0: 0=background, 1=signal)

# División según especificaciones
X_train = X[:2000000] # Primeros 2 millones
y_train = y[:2000000]

X_test = X[-500000:] # Últimos 500 mil
y_test = y[-500000:]

X_extra = X[2000000:-500000] # resto
y_extra = y[2000000:-500000]

print(f"\nDivisión:")
print(f"Train: {X_train.shape}")
print(f"Test:  {X_test.shape}")
print(f"Extra: {X_extra.shape}")

In [ ]:
# Ajustar el scaler SOLO con datos de entrenamiento
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

# Aplicar la misma transformación a test y extra
X_test = scaler.transform(X_test)
X_extra = scaler.transform(X_extra)

print("✓ Normalización completada")
print(f"  - Media train: {X_train.mean():.6f}")
print(f"  - Std train: {X_train.std():.6f}")

# Mostrar distribución de clases
print(f"\n✓ Distribución de clases:")
print(f"  - Train: Signal={np.sum(y_train==1)} ({100*np.mean(y_train==1):.1f}%), "
      f"Background={np.sum(y_train==0)} ({100*np.mean(y_train==0):.1f}%)")
print(f"  - Test:  Signal={np.sum(y_test==1)} ({100*np.mean(y_test==1):.1f}%), "
      f"Background={np.sum(y_test==0)} ({100*np.mean(y_test==0):.1f}%)")